# Phase 2: Profile Loader 사용 예제

Phase 2에서 구현한 Profile Loader의 사용법을 설명합니다.

## 구현된 기능
- BaseProfileLoader (Template Method Pattern)
- ToyoRateProfileLoader (Toyo Rate 테스트 구현)
- 7단계 파이프라인 (용량 계산 → 데이터 로드 → ... → 포맷 변환)

## 1. 기본 사용법

Toyo Rate Profile 데이터를 로드하는 가장 간단한 방법입니다.

In [ ]:
import sys
sys.path.insert(0, '..')

from src.core.toyo_loader import ToyoRateProfileLoader
from src.utils.config_models import ProfileConfig

# 설정 생성
config = ProfileConfig(
    raw_file_path="Rawdata/Q7M Sub ATL [45v 2068mAh] [23] - 250219/30",
    mincapacity=0,  # 자동 계산
    firstCrate=0.2
)

# Profile Loader 생성 및 실행
loader = ToyoRateProfileLoader(config)
result = loader.load()

print("Profile Load Result:")
print(f"  Capacity: {result.mincapacity:.1f} mAh")
print(f"  Data Points: {len(result.data)}")
print(f"\nFirst 5 rows:")
print(result.data.head())

## 2. ProfileData 메타데이터

로드된 프로파일의 메타데이터를 확인합니다.

In [ ]:
# ProfileData 메타데이터
profile_data = result.profile_data

print("Profile Metadata:")
print(f"  Battery Capacity: {profile_data.capacity_mah:.1f} mAh")
print(f"  Nominal Voltage: {profile_data.voltage_v:.2f} V")
print(f"  Chemistry: {profile_data.chemistry}")
print(f"  Manufacturer: {profile_data.manufacturer}")
print(f"  Test Temperature: {profile_data.test_temperature_c}°C")

## 3. DataFrame 구조

로드된 데이터의 구조를 탐색합니다.

In [ ]:
# DataFrame 정보
print("DataFrame Info:")
print(f"  Shape: {result.data.shape}")
print(f"  Columns: {list(result.data.columns)}")
print(f"\nData Types:")
print(result.data.dtypes)

In [ ]:
# 통계 요약
print("\nStatistical Summary:")
print(result.data.describe())

## 4. 데이터 시각화

C-rate별 용량 변화를 시각화합니다.

In [ ]:
import matplotlib.pyplot as plt

# C-rate vs Discharge Capacity
plt.figure(figsize=(10, 6))
plt.plot(result.data['Crate'], result.data['DchgCap'], 'o-', linewidth=2, markersize=8)
plt.xlabel('C-rate', fontsize=12)
plt.ylabel('Discharge Capacity (mAh)', fontsize=12)
plt.title(f'Rate Performance Test - {profile_data.capacity_mah:.0f}mAh Battery', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nCapacity Retention at 2C: {result.data[result.data['Crate']==2.0]['DchgCap'].values[0]:.1f} mAh")

## 5. 수동 용량 지정

자동 계산 대신 수동으로 용량을 지정할 수 있습니다.

In [ ]:
# 수동 용량 지정
config_manual = ProfileConfig(
    raw_file_path="Rawdata/Q7M Sub ATL [45v 2068mAh] [23] - 250219/30",
    mincapacity=2000.0,  # 수동 지정
    firstCrate=0.2
)

loader_manual = ToyoRateProfileLoader(config_manual)
result_manual = loader_manual.load()

print("Manual Capacity Specification:")
print(f"  Specified: {config_manual.mincapacity} mAh")
print(f"  Used: {result_manual.mincapacity} mAh")
print(f"  Match: {config_manual.mincapacity == result_manual.mincapacity}")

## 6. 경로 자동 감지 통합

Phase 1의 Cycler Detector와 통합하여 사용합니다.

In [ ]:
from src.core.cycler_detector import detect_cycler_type, get_channel_folders

# 자동으로 채널 검색 및 로드
base_path = "Rawdata/Q7M Sub ATL [45v 2068mAh] [23] - 250219"
cycler_type = detect_cycler_type(base_path)
channels = get_channel_folders(base_path)

print(f"Detected Cycler: {cycler_type}")
print(f"Found Channels: {channels}")

# 각 채널에 대해 Profile 로드
for channel in channels:
    channel_path = f"{base_path}/{channel}"
    config = ProfileConfig(
        raw_file_path=channel_path,
        mincapacity=0,
        firstCrate=0.2
    )
    
    loader = ToyoRateProfileLoader(config)
    result = loader.load()
    
    print(f"\nChannel {channel}:")
    print(f"  Capacity: {result.mincapacity:.1f} mAh")
    print(f"  Data Points: {len(result.data)}")

## 7. Template Method Pattern 확인

내부적으로 7단계 파이프라인이 실행됩니다.

In [ ]:
# 7단계 파이프라인 단계별 확인
config = ProfileConfig(
    raw_file_path="Rawdata/Q7M Sub ATL [45v 2068mAh] [23] - 250219/30",
    mincapacity=0,
    firstCrate=0.2
)

loader = ToyoRateProfileLoader(config)

print("7-Step Pipeline:")
print("  1. _calculate_capacity() - 용량 계산")
print("  2. _load_raw_data() - Raw 데이터 로드")
print("  3. _filter_data() - 데이터 필터링 (Condition == 1)")
print("  4. _normalize_capacity() - 용량 정규화")
print("  5. _calculate_metrics() - C-rate 계산")
print("  6. _extract_metadata() - 메타데이터 추출")
print("  7. _format_output() - DataFrame 포맷 변환")
print("\nExecuting pipeline...")

result = loader.load()
print(f"\n✓ Pipeline completed successfully")
print(f"  Final capacity: {result.mincapacity:.1f} mAh")
print(f"  Final data points: {len(result.data)}")

## 요약

Phase 2 Profile Loader는:

1. **Template Method Pattern**: 7단계 파이프라인으로 일관된 처리
2. **자동 용량 계산**: 경로명 또는 첫 사이클에서 추출
3. **메타데이터 제공**: ProfileData 객체로 배터리 정보 제공
4. **DataFrame 출력**: Pandas DataFrame으로 쉬운 분석
5. **확장 가능**: 새로운 Cycler 타입 추가 용이 (PNE 등)

다음 Phase에서는 이 데이터를 Database에 저장하고 쿼리하는 기능을 추가합니다.